# Numpy

In [ ]:
import numpy as np
import math

In [ ]:
# random giriş ve random çıkışlarımızı üretelim.

x = np.linspace(-math.pi , math.pi, 200)
y = np.sin(x)

In [ ]:
# sinir ağımızın ağırklıklarını random olarak başlatalım.

a = np.random.randn()
b = np.random.randn()
c = np.random.randn()
d = np.random.randn()

In [ ]:
# learning rate ve eğitim döngümüzü ayarlayalım.

learning_rate = 1e-6
epochs = 2000
for t in range(epochs):
  #forward: tahmin edilen y'yi hesaplayalım
  # y = a + b x + c x^2 + d x^3
  y_pred = a + b * x + c * x ** 2 + d * x ** 3

  # tahmin ve gerçek değer arasındaki kaybı hesaplayalım
  loss = np.square(y_pred - y).sum()
  if t % 100 == 99:
    print(t, loss)

  # backward : kayba göre a, b, c, d gradyanlarını hesaplamak için Backprop yazalım

  grad_y_pred = 2.0 * (y_pred - y)
  grad_a = grad_y_pred.sum()
  grad_b = (grad_y_pred * x).sum()
  grad_c = (grad_y_pred * x ** 2).sum()
  grad_d = (grad_y_pred * x ** 3).sum()

  # hesaplanan gradyanlar ve learning rate kullanrarak ağırlıkları güncelleyelim.
  a -= learning_rate * grad_a
  b -= learning_rate * grad_b
  c -= learning_rate * grad_c
  d -= learning_rate * grad_d

print(f'sonuçlar: y = {a} + {b} x + {c} x^2 + {d} x^3')


# Tensor

In [ ]:
# -*- coding: utf-8 -*-
import numpy as np
import math

# random şekilde giriş ve çıkış değerlerini oluşturalım.
x = np.linspace(-math.pi, math.pi, 2000)
y = np.sin(x)

# ağırlıkları random olarak başlatalım.
a = np.random.randn()
b = np.random.randn()
c = np.random.randn()
d = np.random.randn()

learning_rate = 1e-6
for t in range(2000):
    #forward: tahmin edilen y'yi hesaplayalım
    # y = a + b x + c x^2 + d x^3
    y_pred = a + b * x + c * x ** 2 + d * x ** 3

    # kaybı hesaplayalım.
    loss = np.square(y_pred - y).sum()
    if t % 100 == 99:
        print(t, loss)

    # backward : kayba göre a, b, c, d gradyanlarını hesaplamak için Backprop yazalım
    grad_y_pred = 2.0 * (y_pred - y)
    grad_a = grad_y_pred.sum()
    grad_b = (grad_y_pred * x).sum()
    grad_c = (grad_y_pred * x ** 2).sum()
    grad_d = (grad_y_pred * x ** 3).sum()

    # ağırlıkları güncelleyelim.
    a -= learning_rate * grad_a
    b -= learning_rate * grad_b
    c -= learning_rate * grad_c
    d -= learning_rate * grad_d

print(f'Sonuçlar: y = {a} + {b} x + {c} x^2 + {d} x^3')

Yukarıdaki örneklerimizde sinir ağımızın hem ileri hemde geri geçişlerini manuel olarak uyguladık. Özellikle geriye yayılımı manuel olarak uygulamak küçük networklerde sorun değil ancak büyük ağ mimarilerinde çok sorun olacaktır.

> Bu problemi çözebilmek adına pytorch da ki autograd paketi bu işlemi bizim için yapar.



# nn module

In [ ]:
# -*- coding: utf-8 -*-
import torch
import math


# giriş ve çıkış değerlerimizi ayarlayalım.
x = torch.linspace(-math.pi, math.pi, 2000)
y = torch.sin(x)

# örnekte y çıkışı doğrusal bir fonksiyondur. bu nedenle bunu doğrusal katmanlı
# bir sinir ağı olarak düşünebiliriz.
p = torch.tensor([1, 2, 3])
xx = x.unsqueeze(-1).pow(p)


# modelimizi katmanlar dizisi olarak tanımlamak için: nn.Sequential
# nn.Sequential diğer modülleri içeren ve bunları sırayla uygulayan bir modüldür.
# modül giriş değerinden çıkış değerlerini üretirken ağırlık ve bias değerini
# içeren tensörleri tutar.
# flatten layer doğrusal katmanın çıktısını 1D tensörüne çevirir.
model = torch.nn.Sequential(
    torch.nn.Linear(3, 1),
    torch.nn.Flatten(0, 1)
)

# nn paketi ayrıca popüler kayıp işlevlerini de barındırır.
loss_fn = torch.nn.MSELoss(reduction='sum')

learning_rate = 1e-6
for t in range(2000):

    # forward geçişi sayesinde giriş olarak verilen tensörlerden
    # çıkış tensörleri üretilir.
    y_pred = model(xx)

    # aynı şekilde loss ve y değerleri bir tensördür.
    loss = loss_fn(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())

    # backpro' yu çalıştırnadan önceden gradyanları sıfırlayalım.
    model.zero_grad()

    # Backward pass: kayba göre gradyanları hesaplıyoruz.
    # requires_grad=True olduğunda tüm öğrenilebilir parametreler için kayba göre
    # gradyanlar hesaplanır.
    # her modülün parametreleri saklanır.
    loss.backward()

    # gradyan iniş kullanılarak parametreler güncellenir. her parametre bir 
    # tensördür.

    with torch.no_grad():
        for param in model.parameters():
            param -= learning_rate * param.grad

# Bir listenin ilk öğesine erişmek gibi `modelin' ilk katmanına erişebilirsiniz.
linear_layer = model[0]

print(f'Result: y = {linear_layer.bias.item()} + {linear_layer.weight[:, 0].item()} x + {linear_layer.weight[:, 1].item()} x^2 + {linear_layer.weight[:, 2].item()} x^3')

99 1358.1866455078125
199 905.281494140625
299 604.5720825195312
399 404.86029052734375
499 272.1868591308594
599 184.0220947265625
699 125.41560363769531
799 86.44464111328125
899 60.52105712890625
999 43.27019119262695
1099 31.785930633544922
1199 24.13736343383789
1299 19.04107666015625
1399 15.643819808959961
1499 13.378094673156738
1599 11.866195678710938
1699 10.856791496276855
1799 10.182450294494629
1899 9.731709480285645
1999 9.430209159851074
Result: y = -0.011912121437489986 + 0.8353021740913391 x + 0.002055038930848241 x^2 + -0.0902809202671051 x^3


# optim

Bu noktaya kadar, öğrenilebilir parametreleri tutan Tensörleri **torch.no_grad()** ile manuel olarak değiştirerek modellerimizin ağırlıklarını güncelledik. Bu, stokastik gradyan inişi gibi basit optimizasyon algoritmaları için büyük bir yük değildir


> **optim paketi**, bir optimizasyon algoritması fikrini özetler ve yaygın olarak kullanılan optimizasyon algoritmalarının uygulamalarını sağlar.


